In [2]:
import os
from pathlib import Path
import json
import numpy as np
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import torch.nn.functional as F


Load the np arrays (not on github b/c too big)

In [3]:
small_array = np.load('small_rnn.npy')
multitask_array = np.load('multitask_rnn.npy')

Navigating Array
Array dims: (trial, timestep, data)

In [6]:
print(multitask_array.shape)
print(small_array.shape)

(10000, 100, 311)
(10000, 10, 55)


In [10]:
# Access 1 trial:
print(small_array[0].shape) 
# Access 1 timestep of the trial:
print(small_array[0, 0].shape) 


(10, 55)
(55,)


Unpacking trial data:
data = inputs, label, predicted, hidden states
so for the small 1 task rnn with input size 33, label size 1, prediction size 1, and 20 units/hidden states

In [11]:
data = small_array[0, 0]
inputs = data[:33]
label = data[33:34]
prediction = data[34:35]
hidden_states = data[35:]

For large multitask rnn with input size 53 (33 normal inputs + 20 one hot encoded task vectors), label size 1, prediction size 1, and 256 units/hidden states

In [12]:
data = multitask_array[0, 0]
inputs = data[:53] # fixation input then 2 size 16 rings, then 20 one hot encoded task vector
label = data[53:54]
prediction = data[54:55]
hidden_states = data[55:]